# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

## imports for Python, Pandas

In [111]:
import pandas as pd
import json
from pandas.io.json import json_normalize

# set pandas to display more chars in a column cell
pd.options.display.max_colwidth = 1000

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

name  population
0        Dade       12345
1     Broward       40000
2  Palm Beach       60000
3      Summit        1234
4    Cuyahoga        1337

In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

name  population info.governor    state shortname
0        Dade       12345    Rick Scott  Florida        FL
1     Broward       40000    Rick Scott  Florida        FL
2  Palm Beach       60000    Rick Scott  Florida        FL
3      Summit        1234   John Kasich     Ohio        OH
4    Cuyahoga        1337   John Kasich     Ohio        OH

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{u'_id': {u'$oid': u'52b213b38594d8a2be17c780'},
  u'approvalfy': 1999,
  u'board_approval_month': u'November',
  u'boardapprovaldate': u'2013-11-12T00:00:00Z',
  u'borrower': u'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  u'closingdate': u'2018-07-07T00:00:00Z',
  u'country_namecode': u'Federal Democratic Republic of Ethiopia!$!ET',
  u'countrycode': u'ET',
  u'countryname': u'Federal Democratic Republic of Ethiopia',
  u'countryshortname': u'Ethiopia',
  u'docty': u'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  u'envassesmentcategorycode': u'C',
  u'grantamt': 0,
  u'ibrdcommamt': 0,
  u'id': u'P129828',
  u'idacommamt': 130000000,
  u'impagency': u'MINISTRY OF EDUCATION',
  u'lendinginstr': u'Investment Project Financing',
  u'lendinginstrtype': u'IN',
  u'lendprojectcost': 550000000,
  u'majorsector_percent': [{u'Name': u'Education', u'Percent': 46},
   {u'Name': u'Education', u'Percent': 26},
   {u'Name': u'Public Administration, Law, and Ju

In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

_id  approvalfy board_approval_month  \
0  {u'$oid': u'52b213b38594d8a2be17c780'}        1999             November   
1  {u'$oid': u'52b213b38594d8a2be17c781'}        2015             November   

      boardapprovaldate                                 borrower  \
0  2013-11-12T00:00:00Z  FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA   
1  2013-11-04T00:00:00Z                    GOVERNMENT OF TUNISIA   

            closingdate                              country_namecode  \
0  2018-07-07T00:00:00Z  Federal Democratic Republic of Ethiopia!$!ET   
1                   NaN                      Republic of Tunisia!$!TN   

  countrycode                              countryname countryshortname  \
0          ET  Federal Democratic Republic of Ethiopia         Ethiopia   
1          TN                      Republic of Tunisia          Tunisia   

                         ...                           sectorcode source  \
0                        ...                          ET,BS,ES,EP   IBRD   
1                        ...                                BZ,BS   IBRD   

   status  supplementprojectflg  \
0  Active                     N   
1  Active                     N   

                                              theme1  \
0   {u'Percent': 100, u'Name': u'Education for all'}   
1  {u'Percent': 30, u'Name': u'Other economic man...   

                                      theme_namecode themecode   totalamt  \
0  [{u'code': u'65', u'name': u'Education for all'}]        65  130000000   
1  [{u'code': u'24', u'name': u'Other economic ma...     54,24          0   

  totalcommamt                                                url  
0    130000000  http://www.worldbank.org/projects/P129828/ethi...  
1      4700000  http://www.worldbank.org/projects/P144674?lang=en  

[2 rows x 50 columns]

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

### Get Data

In [28]:
# get the data - read into pandas df
df = pd.read_json('data/world_bank_projects.json')
print df.shape
df.describe()

# also load in the raw json to wrangle the nested fields later
with open('data/world_bank_projects.json') as f:
    raw = json.load(f)

(500, 50)


### Q1:  Get top 10 countries

In [29]:
# group by country and count distinct on id's
df.groupby('country_namecode').id.nunique().sort_values(ascending=False).head(10)

country_namecode
People's Republic of China!$!CN         19
Republic of Indonesia!$!ID              19
Socialist Republic of Vietnam!$!VN      17
Republic of India!$!IN                  16
Republic of Yemen!$!RY                  13
Nepal!$!NP                              12
People's Republic of Bangladesh!$!BD    12
Kingdom of Morocco!$!MA                 12
Africa!$!3A                             11
Republic of Mozambique!$!MZ             11
Name: id, dtype: int64

### Q2: Get top 10 themes

In [33]:
# seems like a one to many relationship between themes (name and code) and id
df_themes = json_normalize(raw, 'mjtheme_namecode', ['id'])
# some projects seem to have multiple theme code's
df_themes.groupby('id').code.nunique().sort_values(ascending=False).head(10)
# so to find top themes we will account for this one to many relationship
print 'Top 10 Major World Bank Project Themes:'
df_themes.name.value_counts().head(10)
# looks like [name] missing for some rows

Top 10 Major World Bank Project Themes:


Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

### Q3: Deal with missing [name] values in [mjtheme_namecode] fields

In [76]:
# create lookup table from code to name
df_themes_name_to_code = df_themes.groupby('name').code.max().sort_values(ascending=False)
# drop the missing name rows
df_themes_name_to_code = df_themes_name_to_code[df_themes_name_to_code.index != '']
# convert to df
df_themes_name_to_code = pd.DataFrame(df_themes_name_to_code,columns=['code'])
# pull name into a column
df_themes_name_to_code['name_clean'] = df_themes_name_to_code.index
# set code to be the index
df_themes_code_to_name = df_themes_name_to_code.set_index(['code'])

# now merge on the name based on the code for the missing projects
print df_themes.shape
df_themes_clean = df_themes.merge(df_themes_code_to_name,how='outer',left_on=['code'],right_index=True)
print df_themes_clean.shape

Now get top 10 themes by name after filling in the missing names based on the theme code

In [87]:
# based on pre cleaned data
print 'Top 10 Major World Bank Project Themes (Original):'
print df_themes.name.value_counts().head(10)
print '--------------------------------------------------'

# based on cleaning we have done
print 'Top 10 Major World Bank Project Themes (Cleaned):'
print df_themes_clean.name_clean.value_counts().head(10)
print '--------------------------------------------------'

Top 10 Major World Bank Project Themes (Original):
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64
--------------------------------------------------
Top 10 Major World Bank Project Themes (Cleaned):
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development   

#### Collapse cleaned data and merge back to original data

In [96]:
# get a list of cleaned theme names by id 
df_theme_names = pd.DataFrame(df_themes_clean.groupby('id').apply(lambda x: '|'.join(x['name_clean'])),columns=['theme_names'])
# get s list of theme codes by id
df_theme_codes = pd.DataFrame(df_themes.groupby('id').apply(lambda x: '|'.join(x['code'])),columns=['theme_codes'])
# now make an id level lookup table thsat can be used later to merge to original data
df_theme_lut = df_theme_names.merge(df_theme_codes,left_index=True,right_index=True)
# look at shape
print df_theme_lut.shape
# take a look
df_theme_lut.head()

(500, 2)


theme_names  theme_codes
id                                                                     
P075941  Trade and integration|Social dev/gender/inclus...     4|7|10|5
P085621  Environment and natural resources management|E...   11|7|11|11
P086592  Environment and natural resources management|R...  11|10|10|10
P094183  Trade and integration|Trade and integration|Ru...       10|5|5
P095003            Trade and integration|Rural development         5|10

Now finally merge collapsed "|" seperated list of theme codes and names for each row in original data

In [113]:
# merge back out cleaned fields into the original data
df_cleaned = df.merge(df_theme_lut,how='right',left_on=['id'],right_index=True,)

# make sure shape has has not changed
print df.shape
print df_cleaned.shape
# we expect to just see the addition of 2 cols

# take a look at the fields to see the cleaning we have done
df_cleaned[['id','mjtheme_namecode','theme_names','theme_codes']].head(10)

(500, 50)
(500, 52)


id  \
0  P129828   
1  P144674   
2  P145310   
3  P144665   
4  P144933   
5  P146161   
6  P121185   
7  P127033   
8  P130164   
9  P130903   

                                                                                                                                                                                                                                              mjtheme_namecode  \
0                                                                                                                                                                             [{u'code': u'8', u'name': u'Human development'}, {u'code': u'11', u'name': u''}]   
1                                                                                                                                       [{u'code': u'1', u'name': u'Economic management'}, {u'code': u'6', u'name': u'Social protection and risk management'}]   
2  [{u'code': u'5', u'name': u'Trade and integration'}, {u'code': u'2', u'name': u'Public sector governance'}, {u'code': u'11', u'name': u'Environment and natural resources management'}, {u'code': u'6', u'name': u'Social protection and risk management'}]   
3                                                                                                                                         [{u'code': u'7', u'name': u'Social dev/gender/inclusion'}, {u'code': u'7', u'name': u'Social dev/gender/inclusion'}]   
4                                                                                                                                  [{u'code': u'5', u'name': u'Trade and integration'}, {u'code': u'4', u'name': u'Financial and private sector development'}]   
5                                                                                                                                                          [{u'code': u'6', u'name': u'Social protection and risk management'}, {u'code': u'6', u'name': u''}]   
6                                                                                                                               [{u'code': u'2', u'name': u'Public sector governance'}, {u'code': u'4', u'name': u'Financial and private sector development'}]   
7                                                                                                                                                  [{u'code': u'11', u'name': u'Environment and natural resources management'}, {u'code': u'8', u'name': u''}]   
8                                                                                                                                                                             [{u'code': u'10', u'name': u'Rural development'}, {u'code': u'7', u'name': u''}]   
9                                                                                        [{u'code': u'2', u'name': u'Public sector governance'}, {u'code': u'2', u'name': u'Public sector governance'}, {u'code': u'2', u'name': u'Public sector governance'}]   

                                                                                                                         theme_names  \
0                                                                     Human development|Environment and natural resources management   
1                                                                          Economic management|Social protection and risk management   
2  Environment and natural resources management|Social protection and risk management|Trade and integration|Public sector governance   
3                                                                            Social dev/gender/inclusion|Social dev/gender/inclusion   
4                                                                     Trade and integration|Financial and private sector development   
5                                                        Social protection and risk management|Social protection and risk management   
6                                                             